# Assistant-03<br/>Function calling - Trading Assistant

## Credits

- https://mer.vin/2023/11/openai-assistants-api-function-calling/

### Get an OpenAI client and an Assistant

**Note:** When configuring the OpenAI client, the API version, the model version, and endpoint are "new".

In [1]:
import common
import time
import oaihelper as framework
import yfinance as yf

## Get an OpenAI client
client = framework.get_openai_client()

### Retreive the latest stock price using Yahoo Finance

In [2]:
def get_stock_price(symbol: str) -> float:
    stock = yf.Ticker(symbol)
    price = stock.history(period="1d")['Close'].iloc[-1]
    return price

get_stock_price("MSFT")

388.4700012207031

### Prepare the tool for function calling

In [3]:
tools_list = [{
    "type": "function",
    "function": {

        "name": "get_stock_price",
        "description": "Retrieve the latest closing price of a stock using its ticker symbol",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The ticker symbol of the stock"
                }
            },
            "required": ["symbol"]
        }
    }
}]

### Create the assistant

In [4]:
framework.clear_shelves()

## Create an assistant
stock_assistant = framework.create_assistant(client,
                                                  name="fta-Securities Trading Assistant",
                                                  instructions="You are a personal securities trading assistant. Please be polite, professional, helpful, and friendly.", 
                                                  tools=tools_list, 
                                                  model=common.gpt_deployment_name)

Added assistant:  asst_wZeuCGaPVSTMJ7iNweMzZmD4 1


### Create a thread, put a message on the thread, and view the initial status

In [5]:
# Step 2: Create a Thread
thread = client.beta.threads.create()

# Step 3: Add a Message to a Thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Can you please provide me the latest stock prices for Microsoft and NVIDIA?"
)

# Step 4: Run the Assistant
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=stock_assistant.id,
    instructions="Please address the user as Mary."
)

print(run.model_dump_json(indent=4))

{
    "id": "run_T4tZgewCziZPDcsLjP013cNS",
    "assistant_id": "asst_wZeuCGaPVSTMJ7iNweMzZmD4",
    "cancelled_at": null,
    "completed_at": null,
    "created_at": 1705172417,
    "expires_at": 1705173017,
    "failed_at": null,
    "file_ids": [],
    "instructions": "Please address the user as Mary.",
    "last_error": null,
    "metadata": {},
    "model": "gpt-4-turbo-test",
    "object": "thread.run",
    "required_action": null,
    "started_at": null,
    "status": "queued",
    "thread_id": "thread_ePQaOVw8V6qebzUYQ4MtVjMc",
    "tools": [
        {
            "function": {
                "name": "get_stock_price",
                "description": "Retrieve the latest closing price of a stock using its ticker symbol",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "symbol": {
                            "type": "string",
                            "description": "The ticker symbol of the stoc

### Process the message, function calling leveraging the latest stock prices, print the results and dispose of the objects

In [6]:
while True:
    # Wait for 5 seconds
    time.sleep(1)

    # Retrieve the run status
    run_status = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    print(run_status.model_dump_json(indent=4))

    # If run is completed, get messages
    if run_status.status == 'completed':
        messages = client.beta.threads.messages.list(
            thread_id=thread.id
        )

        # Loop through messages and print content based on role
        for msg in messages.data:
            role = msg.role
            content = msg.content[0].text.value
            print(f"{role.capitalize()}: {content}")

        ## Cleanup
        client.beta.threads.delete(thread.id)
        client.beta.assistants.delete(stock_assistant.id)

        break
    elif run_status.status == 'requires_action':
        print("Function Calling")
        required_actions = run_status.required_action.submit_tool_outputs.model_dump()
        print(required_actions)
        tool_outputs = []
        import json
        for action in required_actions["tool_calls"]:
            func_name = action['function']['name']
            arguments = json.loads(action['function']['arguments'])
            
            if func_name == "get_stock_price":
                output = get_stock_price(symbol=arguments['symbol'])
                tool_outputs.append({
                    "tool_call_id": action['id'],
                    "output": output
                })
            else:
                raise ValueError(f"Unknown function: {func_name}")
            
        print("Submitting outputs back to the Assistant...")
        client.beta.threads.runs.submit_tool_outputs(
            thread_id=thread.id,
            run_id=run.id,
            tool_outputs=tool_outputs
        )
    else:
        print("Waiting for the Assistant to process...")
        time.sleep(1)

{
    "id": "run_T4tZgewCziZPDcsLjP013cNS",
    "assistant_id": "asst_wZeuCGaPVSTMJ7iNweMzZmD4",
    "cancelled_at": null,
    "completed_at": null,
    "created_at": 1705172417,
    "expires_at": 1705173017,
    "failed_at": null,
    "file_ids": [],
    "instructions": "Please address the user as Mary.",
    "last_error": null,
    "metadata": {},
    "model": "gpt-4-turbo-test",
    "object": "thread.run",
    "required_action": null,
    "started_at": 1705172417,
    "status": "in_progress",
    "thread_id": "thread_ePQaOVw8V6qebzUYQ4MtVjMc",
    "tools": [
        {
            "function": {
                "name": "get_stock_price",
                "description": "Retrieve the latest closing price of a stock using its ticker symbol",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "symbol": {
                            "type": "string",
                            "description": "The ticker symbol 

In [7]:
# Dispose